# Game Theory

Sander Aarts, 2020 (Based on *Lab 11: Matrix Games* by David Williamson, 2013)

**Objectives**
 - Introduce students to basic concepts in 0-sum 2-person games
 - Give students experience at computing optimal mixed minmax strategies
 - Give students experience at playing games

**Reading:** The data in this lab was found from the following Stats Zone articles: [Article 1](https://www.thestatszone.com/archive/statistical-insight-into-penalty-shootouts-where-to-place-your-penalty-13886) and [Article 2](https://www.thestatszone.com/archive/an-analyses-of-penalty-taking-trends-in-the-champions-league-13654)

**Brief description:** In this lab, we consider penalty kicks in soccer as an example of a simple game. First, you
will be introduced to the rules and setting. Next you will play several rounds of the game against the computer. Finally, you will formally solve this game by describing strategies available to the players and characterizing equilibrium strategies and payoffs.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

In [3]:
# imports -- don't forget to run this cell
from ortools.linear_solver import pywraplp as OR

## Part 1: Introducing and playing the Penalty Kick game

In soccer (football) a penalty kick is awarded agaist the team that commits a punishable foul inside their penalty area. The fouled team chooses one player that is allowed to take one free kick at the opposing team's goal while only the goalkeeper can defend. No other players may interfere, so the penalty kick can be seen as a one-shot game with two players. The kicker wants to kick the ball into the goal; the goalie wants to prevent this from happening. I.e. the two players' goals are diametrically opposed. Moreover, following a kick it typically takes the ball about 0.3 seconds to reach the goal, making it impossible for the goalkeeper to react to the kicker's action; both the kicker and the the goalkeeper must decide what to do without knowing the other player's action.

<br>

<div style="align: left; text-align:center;">
    <img src="carli_lloyd_goal.gif" alt="Carli Lloyd (US) scoring a penalty kick against Germany, 2015" title="Carli Lloyd (US) scoring a penalty kick against Germany, 2015." width="360px" />
    <div class="caption">Figure 1: A penalty kick. The kicker plays right; the goalkeeper left. It's a goal!
    </div>
</div>


<br>

You will play a simple version of the penalty kick game. The rules are as follows:
 - You are on offense. One by one you select different kickers (right or left footed) to shoot penalty kicks
 - Your goal is to make as many goals as possible (maximize number of goals)
 - First, you select either a left- or right-footed kicker. The goalie is informed of which you picked.
 - Afterwards, the computer commits to an action; options are to dive **Left**, **Right**, or stay **Middle**
 - Before knowing the goalie's choice, you commit to kick to the goalie's **Left**, **Right**, or **Middle**.
 - The type of kicker, the goalie's action, and your action jointly determine the probability of a win.
 - A goal / no goal is drawn according to this probability. A goal generates 1 point; a save -1. You want to maximize the total score.
 - The table below shows the scoring probabilities for all possible outcomes.
 
 <br>
 
<div style="align: left; text-align:center;">
    <table style="width:60%">
  <tr>
    <th>Scoring Probability</th>
    <th>LL</th>
    <th>LM</th>
    <th>LR</th>
    <th>ML</th>
    <th>MM</th>
    <th>MR</th>
    <th>RL</th>
    <th>RM</th>
    <th>RR</th>
  </tr>
  <tr>
    <td>Right-footed kicker</td>
    <td>.55</td>
    <td>.65</td>
    <td>.93</td>
    <td>.74</td>
    <td>.60</td>
    <td>.72</td>
    <td>.95</td>
    <td>.73</td>
    <td>.70</td>
  </tr>
  <tr>
    <td>Left-footed kicker</td>
    <td>.67</td>
    <td>.70</td>
    <td>.96</td>
    <td>.74</td>
    <td>.60</td>
    <td>.72</td>
    <td>.87</td>
    <td>.65</td>
    <td>.61</td>
  </tr>
</table>
    <div class="caption">Table 1: The first letter in a column denotes the kicker's choice, the latter the goalie's choice. </div>
</div> 

**Q1:** Play 50 rounds against the computer [HERE](http://engri1101.orie.cornell.edu/penalty_kick_source/penalty_kick_game.html). The computer is not dumb
(or at least, it has not been programmed in a stupid way) and so it follows "fictious play" by tying to play its best action against your past choices. When you finish the run of 50 times, the program will report your overall score.  Try to do as well as you can. (You must trust the software that the goals are determined honestly from the listed probabilities.) How well did you do? 

**A:** <font color='blue'> Answers will vary.</font>

## Part 2: Solving the Penalty Kick game

In this part of the lab, we will try to analyze the game that you have just played. In formulating a 2-person
0-sum game, the first step is to figure out all of the pure strategies (not just the good ones) for each of
the players. Remember, one strategy can be thought of as a complete set of instructions that you can give
someone else, so that he or she plays just as you would until the end of the game (without knowing which
moves will be taken in advance).

**Q2:** Give the six pure strategies that you (the striker) might have played.

**A:** <font color='blue'> You can choose the following pure strategies:
    
 - Pick a right-footed striker and kick Right (RR).
 - Pick a right-footed striker and kick Middle (RM).
 - Pick a right-footed striker and kick Left (RL).
 - Pick a left-footed striker and kick Right (LR).
 - Pick a left-footed striker and kick Middle (LM).
 - Pick a left-footed striker and kick Left (LL).
</font>

The pure strategies for the goalie are a little more complicated. After you pick a striker, their dominant foot is announced. At this point, the goalie only has three pure strategies available; Right, Middle, and Left. Imagine we give the goalie explicit instructions which denote exactly what to do for any possibile action the striker may take.

**Q3:** For what actions from the striker do we need to specify a response in any pure strategy for the goalie?

**A:** <font color='blue'>We must tell the goalie what to do in the case of a left-footed kicker and what to do in the case of a right-footed kicker.</font>

Following the notation from the handout, a pure strategy from the goalie takes the form $([L, i], [R, j])$ read as "If the kicker if left-footed, do $i$. If the kicker is right-footed, do $j$."

**Q4:** What values can $i$ and $j$ take? How many pure stratagies does the goalie have?

**A:** <font color='blue'> Both $i$ and $j$ can take values in ${L, M, R}$. Hence, the goalie has 9 pure strategies.</font>

Now, we break the game down into sub-games to make things easier. We have two games: one for the left-footed kicker and one for the right-footed kicker.

**Q5:** Write down the 3x3 payoff matrices for **both** games.

**A:** <font color='blue'>

<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Right-footed striker</th>
        <th>L</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.55</td>
        <td>.65</td>
        <td>.93</td>
    </tr>
    <tr>
        <td>M</td>
        <td>.74</td>
        <td>.60</td>
        <td>.72</td>
    </tr>
    <tr>
        <td>R</td>
        <td>.95</td>
        <td>.73</td>
        <td>.70</td>
    </tr>
    </table>
        <div class="caption">Table (a): Right-footed striker payoff table. The striker is the row player. </div>
</div>

<br>

<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Left-footed striker</th>
        <th>L</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.67</td>
        <td>.70</td>
        <td>.96</td>
    </tr>
    <tr>
        <td>M</td>
        <td>.74</td>
        <td>.60</td>
        <td>.72</td>
    </tr>
    <tr>
        <td>R</td>
        <td>.87</td>
        <td>.65</td>
        <td>.61</td>
    </tr>
    </table>
        <div class="caption">Table (b): Left-footed striker payoff table. The striker is the row player. </div>
</div> 
    </font>

**Q6:** In each matrix, are any pure strategies dominated by another pure strategy?

**A:** <font color='blue'>No.</font>

In lecture and the handout, you learned about dominance by a pure strategy. However, we can also have dominance by a mixed strategy.

**Q7:** Consider the right-footed striker sub-game. What is the expexcted payoff if the striker does a 50/50 mixed strategy between kicking left ($L$) and kicking right ($R$)? You should give the expected probability of a goal in the case the goalie goes right ($R$), middle ($M$), or left ($L$).

**A:**  <font color='blue'>The expected payoff is $\frac{1}{2}(0.55) + \frac{1}{2}(0.95) = 0.75$ for $L$, $\frac{1}{2}(0.65) + \frac{1}{2}(0.73) = 0.69$ for $M$, and $\frac{1}{2}(0.93) + \frac{1}{2}(0.70) = 0.815$ for $R$ </font>

**Q8:** Does the striker's 50/50 mixed strategy between $L$ and $R$ dominate any pure strategy? If so, which one?

**A:**  <font color='blue'>It dominates the pure strategy in which the striker kicks middle ($M$).</font>

You should find that the striker's pure strategy of kicking middle is dominated. Hence, we can remove it from the payoff matrix as it will never be played.

<br>

<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Right-footed striker</th>
        <th>L</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.55</td>
        <td>.65</td>
        <td>.93</td>
    </tr>
    <tr>
    </tr>
    <tr>
        <td>R</td>
        <td>.95</td>
        <td>.73</td>
        <td>.70</td>
    </tr>
    </table>
        <div class="caption">Table (a): Right-footed striker payoff table. The striker is the row player. </div>
</div>

**Q9:** Consider the goalie in the right-footed striker sub-game. Does a 50/50 mix between left $(L)$ and right ($R$) dominate the pure strategy of middle ($M$)? Why or why not?

**A:** <font color='blue'> No, the goalie wants to minimize the probability of a goal unlike the striker. </font>   

**Q11:** Now consider the left-footed striker sub-game. Eliminate any dominated pure strategies. You should be able to simplify the payoff matrix to a 2x3. (Hint: this payoff matrix simplifies in a similar fashion to the right-footed striker payoff matrix.)

**A:** <font color='blue'> Again, the striker's pure strategy of $M$ is dominated by a 50/50 mix between the other two pure strategies. The simplified payoff matrix is:
    
<br>    
    
<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Left-footed striker</th>
        <th>L</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.67</td>
        <td>.70</td>
        <td>.96</td>
    </tr>
    <tr>
        <td>R</td>
        <td>.87</td>
        <td>.65</td>
        <td>.61</td>
    </tr>
    </table>
        <div class="caption">Table (b): Left-footed striker payoff table. The striker is the row player. </div>
</div> 
    
    
</font>

**Q12:** Let's return to the right-footed striker subgame (with the simplified payoff matrix). Assume the striker kicks left ($L$) with probability $p_L$ and kicks right ($R$) with probability $p_R$. What is the expected payoff associated with each of the goalie's pure strategies ($L$, $M$, $R$)?

**A:** <font color='blue'> ($L$) Payoff is $p_L(0.55) + p_R(0.95)$. 
($M$) Payoffayoff is $p_L(0.65) + p_R(0.73)$.
($R$) Payoffayoff is $p_L(0.93) + p_R(0.70)$. </font>  

**Q13:** Simplify your expressions above so that they just depend on the variable $p_L$. (Hint: What is the sum of $p_L$ and $p_R$?)

<font color='blue'> $p_L(0.55) + p_R(0.95) = p_L(0.55) + (1 - p_L)(0.95) = (0.55)p_L + 0.95 - (0.95)p_L = 0.95 - (0.4)p_L$
    $p_L(0.65) + p_R(0.73) = p_L(0.65) + (1 - p_L)(0.73) =  (0.65)p_L + 0.73 - (0.73)p_L = 0.73 - (0.08)p_L$
   $p_L(0.93) + p_R(0.70) = p_L(0.93) + (1 - p_L)(0.70) = (0.93)p_L + 0.70 - (0.70)p_L = 0.70 + (0.23)p_L$  </font>

**Q14:** As the striker, we want to find the mixed strategy which maximizes the minimum probability we score. Using the graphical method, what values of $p_L$ and $p_R$ acheive this? What is the minimum expected probability we score? ([Demos](https://www.desmos.com/calculator) is a helpful tool!)

**A:** <font color='blue'> $p_L = 0.0968, p_R = 0.9032$. The minimum expected probability we score is $0.7223$ </font>

**Q:** In **Q?**, you found the mixed strategy for the striker that maximized the minimum probability we scored. In solving this graphically, you should have graphed three lines. What did each of these lines correspond to?

**A:** <font color='blue'> Each line corresponds to a different pure strategy from the goalie. </font>

**Q:** The optimal point (where the minimum probability of a score was maximized) lies on the intersection of two of these lines? Which two pure strategies from the goalie do these lines correspond to?

**A:** <font color='blue'> They correspond to the pure strategies of middle ($M$) and ($R$). </font>

<font color='red'> Insert argument indicating why the goalie left strategy can be eliminated.</font>

<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Right-footed striker</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.65</td>
        <td>.93</td>
    </tr>
    <tr>
    </tr>
    <tr>
        <td>R</td>
        <td>.73</td>
        <td>.70</td>
    </tr>
    </table>
        <div class="caption">Table (a): Right-footed striker payoff table. The striker is the row player. </div>
</div>
    

**Q15:** The goalie wants to find the mixed strategy which minimizes the maximum probability we (the striker) score. Assume the striker kicks middle ($M$) with probability $q_M$ and kicks right ($R$) with probability $q_R$. Using the graphical method, what values of $q_M$ and $q_R$ acheive this? What is the maximum expected probability we are scored on? ([Demos](https://www.desmos.com/calculator) is a helpful tool!)

**A:** <font color='blue'>($L$) Payoff is $q_M(0.65) + q_R(0.93)$. ($R$) Payoff is $q_M(0.73) + q_R(0.70)$
    
   $q_M(0.65) + q_R(0.93) = (0.65)q_M + (1 - q_M)(0.93) = 0.93 - q_M(0.28)$
    
   $q_M(0.73) + q_R(0.70) = (0.73)q_M + (1 - q_M)(0.70) = 0.70 + q_M(0.03)$

Hence, $q_M = 0.7419, q_R = 0.2581$. The maximum expected probability we score is $0.7223$.</font>

**Q16:** In **Q14**, you maximized the minimum expected probability of a goal. In **Q15**, you minimized the maximum expected probability of a goal. How did these two values relate. Did you expect this?

**A:** <font color='blue'> They are equal. This is expected by the minimax theorem.</font>

For the left-footed striker subgame, let's use linear programming rather than the graphical method to find a mixed-strategy equilibrium! As a breif reminder, there is a special trick that is helpful in formulating LPs to solve this problem. Let's say we have 2 variable: $x$, $y$ and some constraints on them. We want to maximize the smaller of these two variables. Notationally, we can say $\max_{x,y} \min\{x,y\}$. For example, if $(x,y)= (2,3)$ would have an objective value of 2. The LP below achieves exactly this! (where "..." are the rest of the constraints)

$$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & x \geq z \\
\quad & y \geq z \\
\quad  ...
\end{align*}$$

<div style="align: left; text-align:center;">
    <table style="width:60%">
    <tr>
        <th>Left-footed striker</th>
        <th>L</th>
        <th>M</th>
        <th>R</th>
    </tr>
    <tr>
        <td>L</td>
        <td>.67</td>
        <td>.70</td>
        <td>.96</td>
    </tr>
    <tr>
        <td>R</td>
        <td>.87</td>
        <td>.65</td>
        <td>.61</td>
    </tr>
    </table>
        <div class="caption">Table (b): Left-footed striker payoff table. The striker is the row player. </div>
</div> 

**Q17:** Assume the striker kicks left ($L$) with probability $p_L$ and kicks right ($R$) with probability $p_R$. What is the expected payoff associated with each of the goalie's pure strategies ($L$, $M$, $R$)? (Hint: Remember we are thinking about the left-footed kicker now!)

**A:** <font color='blue'>($L$) Payoff is $p_L(0.67) + p_R(0.87)$. ($M$) Payoff is $p_L(0.70) + p_R(0.65)$. ($R$) Payoff is $p_L(0.96) + p_R(0.61)$.  </font>

**Q18:** As the striker, we want to find the mixed strategy which maximizes the minimum probability we score. Write a linear program to solve this problem.

**A:** <font color='blue'> $$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & p_L(0.67) + p_R(0.87) \geq z \\
\quad & p_L(0.70) + p_R(0.65) \geq z \\
\quad & p_L(0.96) + p_R(0.61) \geq z \\
\quad & p_L + p_R = 1 \\
\quad & p_L, p_R \geq 0 \\
\end{align*}$$ </font>

**Q19:** Implement your linear program from **Q18** as an OR-Tools model and use the cell below to solve it.

In [13]:
# define the model
m = OR.Solver('left-kicker_subgame', OR.Solver.CLP_LINEAR_PROGRAMMING);

# TODO: Define the decision variables.
# m.NumVar(lower_bound, upper_bound, name)
# If you want to use infinity as a bound, you can use m.infinity()

# SOLUTION 
R = m.NumVar(0, m.infinity(), 'R');
L = m.NumVar(0, m.infinity(), 'L');
z = m.NumVar(-m.infinity(), m.infinity(), 'z');

# TODO: Define the objective function.
# m.Minimize() or m.Maximize()

# SOLUTION 
m.Maximize(z);

# TODO: Define the constraints.
# m.Add()

# SOLUTION 
m.Add(0.67*L + 0.87*R >= z);
m.Add(0.70*L + 0.65*R >= z);
m.Add(0.96*L + 0.61*R >= z);
m.Add(L + R == 1);

In [14]:
# Solve and print solution
m.Solve()
print('Objective =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective = 0.694
Solution:
R : 0.11999999999999943
L : 0.8800000000000006
z : 0.694


**Q20:** What is an optimal mixed-strategy equilibrium for the (left-footed) striker? What is the minimum expected probability of a goal for this strategy?

**A:** <font color='blue'> $p_R = 0.12$ and $p_L = 0.88$ The minimum expected probability of a goal for this strategy is $0.694$. </font>

**Q21:** The goalie wants to find the mixed strategy which minimizes the maximum probability we (the striker) score. Assume the goalie goes left ($L$) with probability $q_L$, goes middle with probability $q_M$, and goes right ($R$) with probability $q_R$. Again, formulate a linear program to solve this problem. (Hint: it is just the dual of your previous one!)

In [17]:
# define the model
m = OR.Solver('left-kicker_subgame_goalie', OR.Solver.CLP_LINEAR_PROGRAMMING);

# TODO: Define the decision variables.
# m.NumVar(lower_bound, upper_bound, name)
# If you want to use infinity as a bound, you can use m.infinity()

# SOLUTION 
R = m.NumVar(0, m.infinity(), 'R');
M = m.NumVar(0, m.infinity(), 'M');
L = m.NumVar(0, m.infinity(), 'L');
z = m.NumVar(-m.infinity(), m.infinity(), 'z');

# TODO: Define the objective function.
# m.Minimize() or m.Maximize()

# SOLUTION 
m.Minimize(z);

# TODO: Define the constraints.
# m.Add()

# SOLUTION 
m.Add(0.67*L + 0.70*M + 0.96*R <= z);
m.Add(0.87*L + 0.65*M + 0.61*R <= z);
m.Add(L + M + R == 1);

In [18]:
# Solve and print solution
m.Solve()
print('Objective =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective = 0.694
Solution:
R : 0.0
M : 0.8000000000000002
L : 0.1999999999999998
z : 0.694


**Q22:** What is an optimal mixed-strategy equilibrium for the goalie (with left-footed kicker)? What is the minimum expected probability of a goal for this strategy?

**A:** <font color='blue'> $q_R = 0$, $q_M = 0.8$, and $q_L = 0.2$ The minimum expected probability of a goal for this strategy is $0.694$. </font>

**Q23:** Assume the striker always uses the optimal mixed-strategy equilibrium. Is one type of stiker more likely to score a goal than the other striker?

**A:**  <font color='blue'> Yes, the right-footed striker scores with probability $0.7223 > 0.694$. </font>

**Q24:** Collect your findings from above to compile a <b>complete set of instructions</b> for the players in equilibrium. (Hint: even if the computer always chooses one type of striker, you should specify what the goalkeeper should do against the other type; you already did the computation in part 5).

**A:** <font color='blue'> The instructions are:
    
 - Striker is always right-footed and plays left with probabilities: $p_L = 0.0968$ and $p_R = 0.9032$
 - If right-footed striker, the goalkeeper plays left with probabilities: $q_M = 0.7419$ and $q_R = 0.2581$
 - If left-footed striker, the goalkeeper plays left with probabilities: $q_M = 0.8$ and $q_L = 0.2$
</font>

**Q25:** Does the answer to **Q24** make intuitive sense?

<font color='blue'> Yes, the striker shoots in the dominant foot direction more often and the goalie dives in the dominant foot direction more often.  </font>

**Q26:** Play 50 rounds of the [GAME](http://engri1101.orie.cornell.edu/penalty_kick_source/penalty_kick_game.html) again using the optimal strategy you found. Pay attention to the goalie's choices. Does there strategy reflect what you would expect? Did you do better or worse than the first time you played? Was the score what you expected?

<font color='blue'> I would expect we get a goal ~72% of the time so I score 36 but lose 14 so my overall score is 22. I got lucky and got 26 when I played. The goalie stayed middle the entire time though... </font>

## Bonus

In **Part 2**, we divided the problem into subgames first and then used dominance to simplify the two payoff matrices. However, we could fomulate the following linear program to solve the entire model.

In [6]:
# define the model
m = OR.Solver('left-kicker_subgame', OR.Solver.CLP_LINEAR_PROGRAMMING);

RR = m.NumVar(0, m.infinity(), 'RR');
RM = m.NumVar(0, m.infinity(), 'RM');
RL = m.NumVar(0, m.infinity(), 'RL');
LR = m.NumVar(0, m.infinity(), 'LR');
LM = m.NumVar(0, m.infinity(), 'LM');
LL = m.NumVar(0, m.infinity(), 'LL');
z = m.NumVar(-m.infinity(), m.infinity(), 'z');

m.Maximize(z);

m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.70*LR + 0.72*LM + 0.93*LL >= z); # ([R,R],[L,R])
m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.73*LR + 0.60*LM + 0.65*LL >= z); # ([R,R],[L,M])
m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.95*LR + 0.74*LM + 0.55*LL >= z); # ([R,R],[L,L])
m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.70*LR + 0.72*LM + 0.93*LL >= z); # ([R,M],[L,R])
m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.73*LR + 0.60*LM + 0.65*LL >= z); # ([R,M],[L,M])
m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.95*LR + 0.74*LM + 0.55*LL >= z); # ([R,M],[L,L])
m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.70*LR + 0.72*LM + 0.93*LL >= z); # ([R,L],[L,R])
m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.73*LR + 0.60*LM + 0.65*LL >= z); # ([R,L],[L,M])
m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.95*LR + 0.74*LM + 0.55*LL >= z); # ([R,L],[L,L])
m.Add(RR + RM + RL + LR + LM + LL == 1);

In [7]:
# Solve and print solution
m.Solve()
print('Objective =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective = 0.7222580645161291
Solution:
RR : 0.9032258064516129
RM : 0.0
RL : 0.09677419354838718
LR : 0.0
LM : 0.0
LL : 0.0
z : 0.7222580645161291


In [8]:
# define the model
m = OR.Solver('left-kicker_subgame', OR.Solver.CLP_LINEAR_PROGRAMMING);

RRLR = m.NumVar(0, m.infinity(), 'RRLR');
RRLM = m.NumVar(0, m.infinity(), 'RRLM');
RRLL = m.NumVar(0, m.infinity(), 'RRLL');
RMLR = m.NumVar(0, m.infinity(), 'RMLR');
RMLM = m.NumVar(0, m.infinity(), 'RMLM');
RMLL = m.NumVar(0, m.infinity(), 'RMLL');
RLLR = m.NumVar(0, m.infinity(), 'RLLR');
RLLM = m.NumVar(0, m.infinity(), 'RLLM');
RLLL = m.NumVar(0, m.infinity(), 'RLLL');
z = m.NumVar(-m.infinity(), m.infinity(), 'z');

m.Minimize(z);

m.Add(0.70*RRLR + 0.70*RRLM + 0.70*RRLL + 0.73*RMLR + 0.73*RMLM + 0.73*RMLL + 0.95*RLLR + 0.95*RLLM + 0.95*RLLL <= z);
m.Add(0.72*RRLR + 0.72*RRLM + 0.72*RRLL + 0.60*RMLR + 0.60*RMLM + 0.60*RMLL + 0.74*RLLR + 0.74*RLLM + 0.74*RLLL <= z);
m.Add(0.93*RRLR + 0.93*RRLM + 0.93*RRLL + 0.65*RMLR + 0.65*RMLM + 0.65*RMLL + 0.55*RLLR + 0.55*RLLM + 0.55*RLLL <= z);
m.Add(0.70*RRLR + 0.73*RRLM + 0.95*RRLL + 0.70*RMLR + 0.73*RMLM + 0.95*RMLL + 0.70*RLLR + 0.73*RLLM + 0.95*RLLL <= z);
m.Add(0.72*RRLR + 0.60*RRLM + 0.74*RRLL + 0.72*RMLR + 0.60*RMLM + 0.74*RMLL + 0.72*RLLR + 0.60*RLLM + 0.74*RLLL <= z);
m.Add(0.93*RRLR + 0.65*RRLM + 0.55*RRLL + 0.93*RMLR + 0.65*RMLM + 0.55*RMLL + 0.93*RLLR + 0.65*RLLM + 0.55*RLLL <= z);
m.Add(RRLR + RRLM + RRLL + RMLR + RMLM + RMLL + RLLR + RLLM + RLLL == 1);

In [9]:
# Solve and print solution
m.Solve()
print('Objective =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective = 0.722258064516129
Solution:
RRLR : 0.25806451612903203
RRLM : 0.0
RRLL : 0.0
RMLR : 0.0
RMLM : 0.7419354838709679
RMLL : 0.0
RLLR : 0.0
RLLM : 0.0
RLLL : 0.0
z : 0.722258064516129
